In [1]:
import pandas as pd
import numpy as np

# import Natural Language Toolkit
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# import for regext
# regex adalah rangkaian karakter yang mendefinisikan sebuah pola pencarian
import re
import string

In [3]:
# load dataset energi baru terbarukan
twit_energi_baru_terbarukan = pd.read_csv('data/energi-baru-terbarukan.csv')

# load dataset renewable energy
twit_renewable_energi = pd.read_csv('data/renewable-energy.csv')

# load dataset ebt
twit_ebt = pd.read_csv('data/ebt.csv')

# combine all dataset ke twit
frames = [twit_energi_baru_terbarukan, twit_renewable_energi, twit_ebt]
twit = pd.concat(frames)
twit

,Unnamed: 0,tweet_id,author_name,author_username,author_location,content,content_clean,like_count,retweet_count,reply_count,source,keyword,address,longitude,latitude,province,city,country,created_at
0,0,153619597713608704,MariBerlomba,MariBerlomba,di dalam asa pemuda,"1. lomba poster ilmiah ""energi baru/terbarukan...",NaN,0.0,1.0,NaN,Twitter for BlackBerry®,energi baru terbarukan,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-01 23:32:20
1,1,153857491925610496,vegaaulia,vegaaulia,"Bogor, Indonesia",elaahhh ini si bapak tiba2 ngomong soal energi...,NaN,0.0,NaN,1.0,Mobile Web,energi baru terbarukan,"Bogor, West Java, 16121, Indonesia",106.797242,-6.596299,Jawa Barat,Kelurahan Pabaton,Indonesia,2012-01-02 15:17:39
2,2,154361424154603520,enbater,enbater,Indonesia,masih banyak daerah terpencil membutuhkam list...,NaN,0.0,NaN,NaN,MOTOBLUR,energi baru terbarukan,Indonesia,117.890285,-2.483383,Kalimantan Timur,NaN,Indonesia,2012-01-04 00:40:06
3,3,156397219346518017,imronrosidin,imronrosidin,"ÜT: -6.895087,107.627799",gw kira kalo @provocactive gak tertarik dgn ma...,NaN,0.0,NaN,1.0,UberSocial for BlackBerry,energi baru terbarukan,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-09 15:29:37
4,4,156936339718279168,revozone,revozone,Indonesia,satu lagi yang aneh. inget jatropa? alias miny...,NaN,0.0,NaN,NaN,UberSocial for BlackBerry,energi baru terbarukan,Indonesia,117.890285,-2.483383,Kalimantan Timur,NaN,Indonesia,2012-01-11 03:11:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48121,50668,1542534421219360772,KlikLegalID,KlikLegalID,NaN,bkpm ungkap pengembangan dan investasi ebt di ...,NaN,0.0,NaN,NaN,WordPress.com,ebt,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-30 15:43:59
48122,50669,1542540419677859842,himatemiaunila,himatemiaunila,Bandarlampung - Indonesia,[chemation : heat exchanger]..ada beberapa mac...,NaN,0.0,NaN,NaN,Twitter for Android,ebt,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-30 16:07:49
48123,50670,1542557674704740353,txaltxee,txaltxee,NaN,"@prediz45 💯, kpn yo ebt dadi sumber energi lis...",NaN,0.0,NaN,1.0,Twitter Web App,ebt,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-30 17:16:23
48124,50671,1542592685978320896,mememu123,mememu123,Sumba Tengah,"seminar nasional himatikro 2022. "" aktualisasi...",NaN,0.0,NaN,NaN,Twitter for iPad,ebt,"Sumba Tengah, East Nusa Tenggara, Indonesia",119.666277,-9.593584,Nusa Tenggara Timur,NaN,Indonesia,2022-06-30 19:35:30


In [95]:
# cleaning
pd_tweet = twit

# remove Unnamed: 0
pd_tweet = pd_tweet.drop(columns=['Unnamed: 0'])

# set lowercase content
pd_tweet['content'] = pd_tweet['content'].str.lower()

# set content_clean from content
pd_tweet['content_clean'] = pd_tweet['content']

In [96]:
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

# call function remove tweet special
pd_tweet['content_clean'] = pd_tweet['content_clean'].apply(remove_tweet_special)
pd_tweet['content_clean'].head(5)

0    1. lomba poster ilmiah "energi baru/terbarukan...
1    elaahhh ini si bapak tiba2 ngomong soal energi...
2    masih banyak daerah terpencil membutuhkam list...
3    gw kira kalo gak tertarik dgn materi energi ba...
4    satu lagi yang aneh. inget jatropa? alias miny...
Name: content_clean, dtype: object

In [97]:
#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

# call function remove number
pd_tweet['content_clean'] = pd_tweet['content_clean'].apply(remove_number)
pd_tweet['content_clean'].head(5)

0    . lomba poster ilmiah "energi baru/terbarukan"...
1    elaahhh ini si bapak tiba ngomong soal energi ...
2    masih banyak daerah terpencil membutuhkam list...
3    gw kira kalo gak tertarik dgn materi energi ba...
4    satu lagi yang aneh. inget jatropa? alias miny...
Name: content_clean, dtype: object

In [98]:
#remove punctuation
def remove_punctuation(text):
    return re.sub('\W+',' ', text )

# call function remove tanda baca
pd_tweet['content_clean'] = pd_tweet['content_clean'].apply(remove_punctuation)
pd_tweet['content_clean'].head(5)

0     lomba poster ilmiah energi baru terbarukan de...
1    elaahhh ini si bapak tiba ngomong soal energi ...
2    masih banyak daerah terpencil membutuhkam list...
3    gw kira kalo gak tertarik dgn materi energi ba...
4    satu lagi yang aneh inget jatropa alias minyak...
Name: content_clean, dtype: object

In [99]:
#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

# call function remove text yang mempunyai space awal dan akhir
pd_tweet['content_clean'] = pd_tweet['content_clean'].apply(remove_whitespace_LT)
pd_tweet['content_clean'].head(5)

0    lomba poster ilmiah energi baru terbarukan dea...
1    elaahhh ini si bapak tiba ngomong soal energi ...
2    masih banyak daerah terpencil membutuhkam list...
3    gw kira kalo gak tertarik dgn materi energi ba...
4    satu lagi yang aneh inget jatropa alias minyak...
Name: content_clean, dtype: object

In [100]:
#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

# call function remove multiple space
pd_tweet['content_clean'] = pd_tweet['content_clean'].apply(remove_whitespace_multiple)
pd_tweet['content_clean'].head(5)

0    lomba poster ilmiah energi baru terbarukan dea...
1    elaahhh ini si bapak tiba ngomong soal energi ...
2    masih banyak daerah terpencil membutuhkam list...
3    gw kira kalo gak tertarik dgn materi energi ba...
4    satu lagi yang aneh inget jatropa alias minyak...
Name: content_clean, dtype: object

In [101]:
# remove single char
def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

# remove single char dari content clean
pd_tweet['content_clean'] = pd_tweet['content_clean'].apply(remove_single_char)
pd_tweet['content_clean'].head(5)

0    lomba poster ilmiah energi baru terbarukan dea...
1    elaahhh ini si bapak tiba ngomong soal energi ...
2    masih banyak daerah terpencil membutuhkam list...
3    gw kira kalo gak tertarik dgn materi energi ba...
4    satu lagi yang aneh inget jatropa alias minyak...
Name: content_clean, dtype: object

In [102]:
# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

# set token from content clean
# eg "ini topi budi" menjadi ["ini", "topi", "budi"]
pd_tweet['content_token'] = pd_tweet['content_clean'].apply(word_tokenize_wrapper)
pd_tweet['content_token'].head(5)

0    [lomba, poster, ilmiah, energi, baru, terbaruk...
1    [elaahhh, ini, si, bapak, tiba, ngomong, soal,...
2    [masih, banyak, daerah, terpencil, membutuhkam...
3    [gw, kira, kalo, gak, tertarik, dgn, materi, e...
4    [satu, lagi, yang, aneh, inget, jatropa, alias...
Name: content_token, dtype: object

In [103]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

# function untuk melihat total frekuensi kata yang sama
pd_tweet['tweet_tokens_fdist'] = pd_tweet['content_token'].apply(freqDist_wrapper)
pd_tweet['tweet_tokens_fdist'].head(5)

0    {'lomba': 1, 'poster': 1, 'ilmiah': 1, 'energi...
1    {'elaahhh': 1, 'ini': 1, 'si': 1, 'bapak': 1, ...
2    {'masih': 1, 'banyak': 1, 'daerah': 1, 'terpen...
3    {'gw': 1, 'kira': 1, 'kalo': 1, 'gak': 1, 'ter...
4    {'satu': 1, 'lagi': 1, 'yang': 1, 'aneh': 1, '...
Name: tweet_tokens_fdist, dtype: object

In [105]:
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# load file stopwords
txt_stopword = pd.read_csv("data/stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
list_stopwords

# convert list to dictionary
list_stopwords = set(list_stopwords)

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [106]:
#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

# remove word yang ada di stopwords
pd_tweet['tweet_tokens_WSW'] = pd_tweet['content_token'].apply(stopwords_removal)
pd_tweet['tweet_tokens_WSW'].head(5)

0    [lomba, poster, ilmiah, energi, terbarukan, de...
1    [elaahhh, ngomong, energi, terbarukan, tv, biruu]
2    [daerah, terpencil, membutuhkam, listrik, mema...
3    [tertarik, materi, energi, terbarukan, baek, g...
4    [aneh, inget, jatropa, alias, minyak, jarak, b...
Name: tweet_tokens_WSW, dtype: object

In [107]:
# import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

69684
------------------------
lomba : lomba
poster : poster
ilmiah : ilmiah
energi : energi
terbarukan : baru
deadline : deadline
januari : januari
diperuntukkan : untuk
siswa : siswa
sma : sma
smk : smk
sederajat : derajat
elaahhh : elaahhh
ngomong : ngomong
tv : tv
biruu : biruu
daerah : daerah
terpencil : pencil
membutuhkam : membutuhkam
listrik : listrik
memanfaatkan : manfaat
tertarik : tarik
materi : materi
baek : baek
ganti : ganti
nama : nama
propasif : propasif
ajah : ajah
aneh : aneh
inget : inget
jatropa : jatropa
alias : alias
minyak : minyak
jarak : jarak
bbrp : bbrp
digembar : digembar
gemborkan : gembor
ebt : ebt
eco : eco
power : power
booster : booster
karya : karya
mahasiswa : mahasiswa
universitas : universitas
nasional : nasional
pengembangan : kembang
tawur : tawur
misi : misi
menjalankan : jalan
usaha : usaha
gas : gas
terintegrasi : integrasi
berdasarkan : dasar
prinsip : prinsip
komersial : komersial
kuat : kuat
serunya : seru
adik : adik
sd : sd
pangudi : pang

In [ ]:
term_dict = {}
# convert hasil semua token ke dalam array
# contoh ["ini", "topi", "budi"], ["energi", "baru", "terbarukan"] menjadi ["ini", "topi", "budi", "energi", "baru", "terbarukan"]
for document in pd_tweet['tweet_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

In [ ]:
# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

# replace kata ke bentuk dasarnya berdasarkan sastrawi
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")

In [108]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]
    
# hasil dari sastrawi diubah ke dalam token
pd_tweet['tweet_tokens_stemmed'] = pd_tweet['tweet_tokens_WSW'].swifter.apply(get_stemmed_term)
pd_tweet['tweet_tokens_stemmed'].head(5)

/Users/indralionardy/Personal/sara/tugas-akhir/venv/lib/python3.9/site-packages/swifter/swifter.py:83: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Pandas Apply:   0%|          | 0/109705 [00:00<?, ?it/s]

0    [lomba, poster, ilmiah, energi, baru, deadline...
1          [elaahhh, ngomong, energi, baru, tv, biruu]
2    [daerah, pencil, membutuhkam, listrik, manfaat...
3    [tarik, materi, energi, baru, baek, ganti, nam...
4    [aneh, inget, jatropa, alias, minyak, jarak, b...
Name: tweet_tokens_stemmed, dtype: object

In [109]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
def reverse_token(tokens):
    return TreebankWordDetokenizer().detokenize(tokens)
    
# hasil token yang sudah di kembalikan ke kata dasar oleh sastrawi diubah kembali menjadi kalimat
pd_tweet['tweet_tokens_stemmed_string'] = pd_tweet['tweet_tokens_stemmed'].apply(reverse_token)


In [110]:
# export ke csv dan excel
pd_tweet.to_csv("data/text-preprocessing.csv")
pd_tweet.to_excel("data/text-preprocessing.xlsx")